In [1]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random

In [3]:
from training_data import TRAIN_DATA

In [4]:
# verify train data works
for text, annotation in TRAIN_DATA:
    start_index = annotation["entities"][0][0]
    end_index = annotation["entities"][0][1]
    
    print(text[start_index:end_index])
    
    if len(annotation["entities"]) > 1:
        start_index = annotation["entities"][1][0]
        end_index = annotation["entities"][1][1]
        print(text[start_index:end_index])

repoX.
repoY.
repoZ.
repo123
main
develop
feature/new-feature
hotfix
#general
#development
#marketing
#team-leads
pull requests
repoX.
PR #123
repoY.
PRs
repoZ
PRs
hotfix


In [5]:
# Create a blank English model
nlp = spacy.blank("en")

# Add the NER component
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")

for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])  # Add entity labels (e.g., "REPOSITORY", "BRANCH")

# Create training examples
examples = []
for text, annotations in TRAIN_DATA:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    examples.append(example)

In [7]:
# Start training
optimizer = nlp.begin_training()
for i in range(20):  # Number of iterations
    random.shuffle(examples)
    losses = {}
    batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        nlp.update(batch, drop=0.35, losses=losses)
    print(f"Iteration {i + 1}: Losses - {losses}")

Iteration 1: Losses - {'ner': np.float32(106.84619)}
Iteration 2: Losses - {'ner': np.float32(94.833405)}
Iteration 3: Losses - {'ner': np.float32(65.96361)}
Iteration 4: Losses - {'ner': np.float32(40.45555)}
Iteration 5: Losses - {'ner': np.float32(35.238857)}
Iteration 6: Losses - {'ner': np.float32(35.18705)}
Iteration 7: Losses - {'ner': np.float32(29.717886)}
Iteration 8: Losses - {'ner': np.float32(27.533236)}
Iteration 9: Losses - {'ner': np.float32(25.855633)}
Iteration 10: Losses - {'ner': np.float32(23.798717)}
Iteration 11: Losses - {'ner': np.float32(30.205662)}
Iteration 12: Losses - {'ner': np.float32(23.020939)}
Iteration 13: Losses - {'ner': np.float32(22.922615)}
Iteration 14: Losses - {'ner': np.float32(20.011106)}
Iteration 15: Losses - {'ner': np.float32(16.589085)}
Iteration 16: Losses - {'ner': np.float32(13.470038)}
Iteration 17: Losses - {'ner': np.float32(13.104949)}
Iteration 18: Losses - {'ner': np.float32(10.065937)}
Iteration 19: Losses - {'ner': np.float3

In [8]:
nlp.to_disk("custom_ner_model")